In [11]:
import pandas as pd
import re
import datetime as dt
import numpy as np

In [12]:
bbmp_df = pd.read_json("ChatExport_2021-05-21 (1)/result.json")
burban_df = pd.read_json("ChatExport_2021-05-21/result.json")

bbmp_df['area'] = "BBMP"
burban_df['area'] = "Bengaluru Urban"

df = pd.concat([bbmp_df, burban_df], ignore_index=True)

In [13]:
def extract_date(x):
    return x['date']
df['date'] = df['messages'].apply(extract_date)

def extract_text(x):
    return x['text']
df['text'] = df['messages'].apply(extract_text)

df.drop(['id','name', 'type','messages'], axis=1, inplace=True)

In [14]:
def extract_zip(x):
    
    groups = re.search('(\d{6})', str(x)) 
    
    if groups != None:
        pin = groups.group()
        return pin
    else:
        return 0
    
df['pincode'] = df['text'].apply(extract_zip)
df = df[~(df['pincode']==0)]

In [15]:
df = df[(df['pincode'].str.startswith('56'))]
df['pincode'] = df['pincode'].astype('int')
df['date'] = df['date'].str.replace('T', ' ')

In [16]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
df.dtypes

area               object
date       datetime64[ns]
text               object
pincode             int32
dtype: object

In [17]:
def BBMP(x):
    try: 
        key = x.split('\n').index('BBMP')
    except: 
        pass 
    else: 
        return key

def Bangalore_Rural(x):
    try: 
        key = x.split('\n').index('Bangalore Rural')
    except: 
        pass 
    else: 
        return key

def Bangalore_Urban(x):
    try: 
        key = x.split('\n').index('Bangalore Urban')
    except: 
        pass 
    else: 
        return key

def test(x):   
        key_BBMP = BBMP(x)
        key_BR = Bangalore_Rural(x) 
        key_BU = Bangalore_Urban(x)
    
        if key_BBMP == key_BR == None:
            return key_BU
        elif key_BBMP == key_BU == None:
            return key_BR
        elif key_BR == key_BU == None:
            return key_BBMP

In [18]:
def check_str(x):

    if type(x) == str:
        return x.split('\n')[1].split('-')[0].split('1.')[1].strip()
   
    elif ''.join(x[0]).startswith('Vacc') and len(x)==2:
        return x[0].split('\n')[1].split('- P')[0].split('1.')[1].strip()
    
    elif ''.join(x[0]).startswith('Vacc') and x[4].startswith(' at'):
        return x[4].split('at')[1].split(' on')[0].strip()    
    
    else:
        index = test(x[0])
        if type(index)==int:
            return x[0].split('\n')[index+1]
        else:
            return np.nan

df['Hospital'] = df['text'].apply(check_str)

In [19]:
def ext_slots(x):
    
    if len(re.findall('\d+ slots', str(x)))!=0:
        return re.findall('\d+ slots', str(x))[0].split(' slots')[0]
    
    else: 
        return np.nan

df['slots'] = df['text'].apply(ext_slots)
df

,area,date,text,pincode,Hospital,slots
1,BBMP,2021-05-01 23:35:05,Vaccination centers for 18-45 group: \n1. Shan...,560027,Shanthinagar UPHC BLOCK 1 (East),1
2,BBMP,2021-05-02 17:46:05,Vaccination centers for 18-44 group: \n1. MANI...,560003,MANIPAL NORTHSIDE (West),2
3,BBMP,2021-05-02 18:26:05,Vaccination centers for 18-44 group: \n1. APOL...,560020,APOLLO HOSPITAL SHESHADRIPURAM (West),95
4,BBMP,2021-05-02 19:34:07,Vaccination centers for 18-44 group: \n1. Mani...,560017,Manipal Hospital Covaxin (East),66
6,BBMP,2021-05-02 22:45:01,Vaccination centers for 18-44 group: \n1. Mani...,560017,Manipal Hospital Covaxin (East),90
...,...,...,...,...,...,...
413,Bengaluru Urban,2021-05-10 19:55:04,[Vaccination centers for 18-44 group: \n1. NIM...,560029,NIMHANS Hospitals (Bengaluru South),150
414,Bengaluru Urban,2021-05-10 19:57:04,[Vaccination centers for 18-44 group: \n1. DCH...,560023,DCHC Leprosy Hospital (Bengaluru North),141
415,Bengaluru Urban,2021-05-10 19:59:04,[Vaccination centers for 18-44 group: \n1. Ane...,562106,Anekal TH (Anekal),133
416,Bengaluru Urban,2021-05-10 20:00:05,[Vaccination centers for 18-44 group: \n1. Gen...,560036,General Hospital K R Puram (Bengaluru East),150


In [20]:
df.to_excel('Hospital_vaccine_'+dt.date.today().strftime('%Y-%m-%d')+'.xlsx', index=False, sheet_name='Raw Data')

In [21]:
df['date w/o timestamp'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [22]:
df

,area,date,text,pincode,Hospital,slots,date w/o timestamp
1,BBMP,2021-05-01 23:35:05,Vaccination centers for 18-45 group: \n1. Shan...,560027,Shanthinagar UPHC BLOCK 1 (East),1,2021-05-01 23:35:05
2,BBMP,2021-05-02 17:46:05,Vaccination centers for 18-44 group: \n1. MANI...,560003,MANIPAL NORTHSIDE (West),2,2021-05-02 17:46:05
3,BBMP,2021-05-02 18:26:05,Vaccination centers for 18-44 group: \n1. APOL...,560020,APOLLO HOSPITAL SHESHADRIPURAM (West),95,2021-05-02 18:26:05
4,BBMP,2021-05-02 19:34:07,Vaccination centers for 18-44 group: \n1. Mani...,560017,Manipal Hospital Covaxin (East),66,2021-05-02 19:34:07
6,BBMP,2021-05-02 22:45:01,Vaccination centers for 18-44 group: \n1. Mani...,560017,Manipal Hospital Covaxin (East),90,2021-05-02 22:45:01
...,...,...,...,...,...,...,...
413,Bengaluru Urban,2021-05-10 19:55:04,[Vaccination centers for 18-44 group: \n1. NIM...,560029,NIMHANS Hospitals (Bengaluru South),150,2021-05-10 19:55:04
414,Bengaluru Urban,2021-05-10 19:57:04,[Vaccination centers for 18-44 group: \n1. DCH...,560023,DCHC Leprosy Hospital (Bengaluru North),141,2021-05-10 19:57:04
415,Bengaluru Urban,2021-05-10 19:59:04,[Vaccination centers for 18-44 group: \n1. Ane...,562106,Anekal TH (Anekal),133,2021-05-10 19:59:04
416,Bengaluru Urban,2021-05-10 20:00:05,[Vaccination centers for 18-44 group: \n1. Gen...,560036,General Hospital K R Puram (Bengaluru East),150,2021-05-10 20:00:05
